In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from subprocess import run

In [2]:
## Efficacy Estimator
## Usage: ./bin/EfficacyEstimator [OPTIONS]
#
## Options:
##   -h,--help                   Print this help message and exit
##   -i,--input TEXT             Input filename. Default: `input.yml`
##   -f,--file                   Output to file. Default: false.
##   -c,--console                Output to file. Default: false.
##   -a,--art                    Drug is artemisinin / ACT. Default: false
##   -p,--popsize INT            Number of individuals used in the simulation (default: 10,000)
##   -d,--dosing INT ...         Drung dosing days.
##                               Ex: `-d 2` or `--dosing 2` for monotherapy,
##                                 `-d 5 2` or `--dosing 5 2` for a combination of two drugs.
##   -t,--halflife FLOAT ...     Drug elimintaion half-life in date unit.
##                               Ex: `-t 2` or `--halflife 2` for monotherapy,
##                                 `-t 4.5 28.0` or `--halflife 4.5 28.0` for a combination of two drugs.
##   -k,--kmax FLOAT ...         The maximum fraction of parasites that can be killed per day.
##                               Ex: `-k 0.999` or `--kmax 0.999` for monotherapy,
##                                 `-k 0.999 0.99` or `--kmax= 0.999 0.99` for drug combination.
##   -m,--mda FLOAT ...          Mean drug absorption
##   -e,--EC50 FLOAT ...         The drug concentration at which the parasite killng reach 50%.
##                               Ex: `-e 0.75` or `--EC50 0.75` for monotherapy, 
##                                `-e 0.75 0.65` or `--EC50 0.75 0.65` for drug combination.
##   -n,--slope FLOAT ...        the slope of the concentration-effect curve.
##                               Ex: `-n 25` or `--slope 25` for monotherapy,
##                                `-n 25 10` or `--slope 25 10` for drug combination.

In [3]:
# function to call EfficacyEstimator
def Get_Efficacy(drug_1, drug_2=None, as_or_act=False, input_file=Path(Path.home(), 'Code/BoniLab-MaSim/analysis/dxg/ee_input.yml'), verbose=False):
    # dirpath of the executable
    exe_path = Path(Path.home(), 'Code/BoniLab-MaSim/build/bin/')
    # prepare the command template
    # if only 1 drug
    if drug_2 is None:
        cmd = './EfficacyEstimator -d {dosing_days} -t {halflife} -k {pmax} -m {mda} -e {ec50} -n {slope}'
        if as_or_act:
            cmd += ' -a'
    # if 2 drugs, add the suffix _2 to the keys
    else:
        # iterate over items in drug_2
        for k, v in drug_2.items():
            # add the key and value from drug_2 to drug_1
            drug_1[f'{k}_2'] = v
        # add the suffix _2 to the keys
        cmd = './EfficacyEstimator -d {dosing_days} {dosing_days_2} -t {halflife} {halflife_2} -k {pmax} {pmax_2} -m {mda} {mda_2} -e {ec50} {ec50_2} -n {slope} {slope_2}'
        if as_or_act:
            cmd += ' -a'
    
    cmd = cmd.format_map(drug_1)
    cmd += ' -i {}'.format(input_file)
    
    if verbose:
        print(cmd)

    # run the command from the executable directory
    proc = run(cmd.split(), cwd=exe_path, capture_output=True)
    if proc.returncode == 0:
        # get the output
        output = proc.stdout.decode('utf-8')
    # if unsuccessful
    else:
        print(cmd)
        raise Exception('Error: ', proc.stdout.decode('utf-8'))

    return float(output.split()[-1])

In [13]:
## artemether config
drug_1_conf = {
    'name': 'Artemether',
    'halflife': 0.175,
    'ec50': 0.6,
    'slope': 25,
    'pmax': 0.9999,
    'dosing_days': 3,
    'mda': 1.0
}

## lumefantrine config
drug_2_conf = {
    'name': 'Lumefantrine',
    'halflife': 4.5,
    'ec50': 0.6,
    'slope': 25,
    'pmax': 0.99,
    'dosing_days': 3,
    'mda': 1.0
}

# wrapper function to get efficacy
def Get_Efficacy_Wrapper(ec50_1, ec50_2=None, as_or_act=False):
    # if AS monotherapy (as_or_act=True), use drug_1_conf
    if as_or_act:
        d1 = drug_1_conf.copy()
    # else, use drug_2_conf
    else:
        d1 = drug_2_conf.copy()
    d1['ec50'] = ec50_1
    # if 2 ec50s
    if ec50_2 is not None:
        d2 = drug_2_conf.copy()
        d2['ec50'] = ec50_2
        return Get_Efficacy(d1, d2, as_or_act=as_or_act)
    else:    
        return Get_Efficacy(d1, as_or_act=as_or_act)

In [5]:
Get_Efficacy(drug_1_conf, drug_2_conf)

0.9864

In [7]:
# list of ec50 val
ec50_list = np.linspace(0.01, 1.99, 100)

# get the efficacy for each pair of ec50
# eff_list = [Get_Efficacy_Wrapper(ec50_1, ec50_2, as_or_act=True) for ec50_1 in ec50_list for ec50_2 in ec50_list]

In [8]:
# parallelize the process
from multiprocessing import Pool
from itertools import product

# parallelize the process, as_or_act=True
with Pool(8) as p:
    eff_list = p.starmap(Get_Efficacy_Wrapper, product(ec50_list, ec50_list, [True]))


In [10]:
# combine ec50_1, ec50_2, efficacy into a dataframe
df = pd.DataFrame({'ec50_am': np.repeat(ec50_list, len(ec50_list)), 'ec50_lm': np.tile(ec50_list, len(ec50_list)), 'efficacy': eff_list})
df.head()

,ec50_am,ec50_lm,efficacy
0,0.01,0.01,1.0
1,0.01,0.03,1.0
2,0.01,0.05,1.0
3,0.01,0.07,1.0
4,0.01,0.09,1.0


In [12]:
# save to csv
df.to_csv('al_ec50_efficacy.csv', index=False)

In [14]:
# parallelize the process, AS monotherapy
with Pool(8) as p:
    am_eff_list = p.starmap(Get_Efficacy_Wrapper, product(ec50_list, [None], [True]))

In [15]:
# parallelize the process, LM monotherapy
with Pool(8) as p:
    lm_eff_list = p.starmap(Get_Efficacy_Wrapper, product(ec50_list, [None], [False]))

In [16]:
# combine ec50 and efficacy of monotherapies into a dataframe
df_am = pd.DataFrame({'ec50_am': ec50_list, 'efficacy': am_eff_list})
df_am.to_csv('am_ec50_efficacy.csv', index=False)

df_lm = pd.DataFrame({'ec50_lm': ec50_list, 'efficacy': lm_eff_list})
df_lm.to_csv('lm_ec50_efficacy.csv', index=False)

print(df_am.tail())
df_lm.tail()

    ec50_am  efficacy
95     1.91    0.0058
96     1.93    0.0076
97     1.95    0.0057
98     1.97    0.0048
99     1.99    0.0047


,ec50_lm,efficacy
95,1.91,0.0105
96,1.93,0.0093
97,1.95,0.0077
98,1.97,0.0077
99,1.99,0.0054
